In [ ]:
import os
os.chdir('../daizika')

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from edgar import SECHelper, SECEdgarParser

In [ ]:
!sudo rm -rf /development/data/sec/edgar/extract
!sudo rm -rf /development/data/sec/edgar/forms
!sudo rm -rf /development/data/sec/edgar/index/daily
!sudo aws s3 rm s3://data.daizika.com/sec/edgar/extract/metadata/temp/ --recursive  
start_date = datetime(2018, 3, 27)
end_date = datetime(2018, 4, 1)
current_date = start_date
while current_date < end_date:
    print('Processing: {}'.format(current_date))
    index_date = current_date
    # Download Daily Form Index file
    formDownloader = SECHelper.FormDownloader()
    formDownloader.download_daily_index(index_date) 
    formDownloader.convert_daily_indx_to_csv(index_date)
    formDownloader.download_daily_forms(index_date=index_date, forms=['10-K'])
    print('...done')
    current_date += timedelta(days=1)
formProcessor = SECEdgarParser.FormProcessor()
form_list = formProcessor.extract_documents(form_type='10-K', extract_type=['10-K'], metadata=True)
form_list = formProcessor.extract_text(form_type='10-K')
formProcessor.convert_metadata_to_csv()
!sudo rm /development/data/sec/edgar/index/daily/*.idx
!sudo aws s3 cp /development/data/sec/edgar/index/daily/ s3://data.daizika.com/sec/edgar/index/daily/ --recursive
!sudo aws s3 cp /development/data/sec/edgar/extract/metadata/ s3://data.daizika.com/sec/edgar/extract/metadata/ --recursive    
!sudo aws s3 cp /development/data/sec/edgar/extract/json/ s3://data.daizika.com/sec/edgar/extract/json/ --recursive    
print('Done')

In [ ]:
#Use SQLWorkbench to load  s3://data.daizika.com/sec/edgar/index/daily/form*.csv file to database
#Use SQLWorkbench to load  s3://data.daizika.com/sec/edgar/extract/metadata/temp/metadata.csv file to database
#Use SQLWorkbench to load  s3://data.daizika.com/sec/edgar/extract/metadata/temp/company_name_changes.csv file to database